In [1]:
CPD_URL = "https://cpd-cpd.ai-governance-94074a334e51addd457c5646c0f9a073-0000.us-east.containers.appdomain.cloud"
CPD_USERNAME = "admin"
CPD_PASSWORD = "cp4d@dm1n"

CPD_URL = CPD_URL.rstrip("/")

WOS_CREDENTIALS = {
    "url": CPD_URL,
    "username": CPD_USERNAME,
    "password": CPD_PASSWORD
}

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

PROJECT_ID = "d618d8af-bc48-4582-a228-7f4bcfc327e9"
CONTAINER_ID = PROJECT_ID
CONTAINER_TYPE = 'project'
EXPERIMENT_NAME = 'SciKitExperiment_fact'

In [2]:
import sklearn
sklearn.__version__

'1.1.1'

In [3]:
!pip install ibm-aigov-facts-client --no-cache | tail -n 1

In [4]:
from ibm_aigov_facts_client import AIGovFactsClient, CloudPakforDataConfig

creds=CloudPakforDataConfig(service_url=CPD_URL, username=CPD_USERNAME, password=CPD_PASSWORD, disable_ssl_verification=True)
facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

2023/04/05 19:41:52 INFO : Experiment SciKitExperiment_fact does not exist, creating new experiment
2023/04/05 19:41:52 INFO : Experiment successfully created with ID 1 and name SciKitExperiment_fact
2023/04/05 19:41:52 INFO : Autolog enabled Successfully


In [5]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

1.0.256


Insert the data to code as a pandas dataframe. Note that the 'infer_schema' value is set to false, as we will manually set the datatypes next to avoid mixing types and confusing OpenScale's data consistency drift monitor.

In [6]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'false',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
data_df_1.head(10)


POSITION_CODE DEPARTMENT_CODE DAYS_WITH_COMPANY COMMUTE_TIME  \
0          1200             200              1825           29   
1          1200             200              2615            0   
2          1300             320              1609           30   
3          1300             320              2035           13   
4          1400             380              1885           31   
5          1400             380              3070           35   
6          1400             380              1825            9   
7          1500             300              2035           11   
8          1500             300              2250            0   
9          1600             330              1520           31   

  AGE_BEGIN_PERIOD GENDER_CODE ATTRITION PERIOD_TOTAL_DAYS STARTING_SALARY  \
0               55           0         0               330       159230.77   
1               49           0         0               180       181153.85   
2               44           1         0               330       129692.31   
3               45           0         0               330       146769.23   
4               44           0         0               330       146769.23   
5               40           0         0               330       157846.15   
6               40           1         0               330       146769.23   
7               46           0         0               330       132923.08   
8               45           1         0               180       150461.54   
9               46           0         0               330       143076.92   

  ENDING_SALARY  ... VACATION_DAYS_TAKEN SICK_DAYS_TAKEN PROMOTIONS  \
0     161538.46  ...                  28            10.5          0   
1     183846.15  ...                  15               4          0   
2     132923.08  ...                  20              15          0   
3     150461.54  ...                  28               8          0   
4     150461.54  ...                  26            11.5          0   
5     161538.46  ...                  19            12.5          0   
6     150461.54  ...                  20               9          0   
7     136153.85  ...                  10               9          0   
8     154153.85  ...                  10               4          0   
9     146769.23  ...                  26             9.5          0   

  NB_MANAGERS DAYS_IN_POSITION DAYS_SINCE_LAST_RAISE RANKING_CODE OVERTIME  \
0           1             1825                     0            3        0   
1           1             2615                    60            3        0   
2           1             1609                   150            3        0   
3           1             2035                   210            3        0   
4           1             1885                    60            3        0   
5           1             3070                   150            3        0   
6           1             1825                     0            3        0   
7           1             2035                   210            3        0   
8           1             2250                    60            3        0   
9           1             1520                    60            3        0   

  DBLOVERTIME TRAVEL  
0           0      0  
1           0      0  
2           0      0  
3           0      0  
4           0      0  
5           0      0  
6           0      0  
7           0      0  
8           0      0  
9           0      0  

[10 rows x 23 columns]

In [7]:
data_df_1.dtypes

POSITION_CODE            object
DEPARTMENT_CODE          object
DAYS_WITH_COMPANY        object
COMMUTE_TIME             object
AGE_BEGIN_PERIOD         object
GENDER_CODE              object
ATTRITION                object
PERIOD_TOTAL_DAYS        object
STARTING_SALARY          object
ENDING_SALARY            object
NB_INCREASES             object
BONUS                    object
NB_BONUS                 object
VACATION_DAYS_TAKEN      object
SICK_DAYS_TAKEN          object
PROMOTIONS               object
NB_MANAGERS              object
DAYS_IN_POSITION         object
DAYS_SINCE_LAST_RAISE    object
RANKING_CODE             object
OVERTIME                 object
DBLOVERTIME              object
TRAVEL                   object
dtype: object

In [8]:
import pandas as pd

data_df_1 = data_df_1.apply(pd.to_numeric, errors='coerce').fillna(data_df_1)
data_df_1 = data_df_1.astype({'ATTRITION': 'bool'})

In [9]:
from sklearn.model_selection import train_test_split

X = data_df_1.drop(['ATTRITION'], axis=1)  # Features
y = data_df_1['ATTRITION']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # 85% training and 15% test

In [10]:
fields=X_train.columns.tolist()
metadata_dict = {'target_col' : 'ATTRITION', 'fields':fields}

In [11]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
    wml_client._models.ConfigurationMetaNames.NAME:"{}".format("employee attrition challenger"),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.0",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "ATTRITION",
    wml_client._models.ConfigurationMetaNames.CUSTOM: metadata_dict
}

Software Specification ID: 12b83a17-24d8-5082-900f-0ab31fbfd3cb


In [12]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

2023/04/05 19:42:13 INFO : logging results to factsheet for run_id 64b023d758bc456fa3c5e19a66d09c8f
2023/04/05 19:42:13 INFO : Successfully logged results to Factsheet service for run_id 64b023d758bc456fa3c5e19a66d09c8f under asset_id: e920ab38-e5d4-43d7-8dce-bfc71b9605f0 and project_id : d618d8af-bc48-4582-a228-7f4bcfc327e9


In [13]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
CUSTOM_ACC = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", CUSTOM_ACC)

Accuracy: 0.9166666666666666


In [14]:
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

COMMUTE_TIME             0.202412
OVERTIME                 0.181167
DBLOVERTIME              0.104683
DAYS_IN_POSITION         0.060540
PERIOD_TOTAL_DAYS        0.055103
DAYS_WITH_COMPANY        0.047296
ENDING_SALARY            0.044419
STARTING_SALARY          0.043704
SICK_DAYS_TAKEN          0.040247
DAYS_SINCE_LAST_RAISE    0.037660
AGE_BEGIN_PERIOD         0.037357
RANKING_CODE             0.032860
VACATION_DAYS_TAKEN      0.032722
BONUS                    0.031946
POSITION_CODE            0.022141
DEPARTMENT_CODE          0.013428
GENDER_CODE              0.006912
NB_BONUS                 0.002722
NB_INCREASES             0.002681
NB_MANAGERS              0.000000
PROMOTIONS               0.000000
TRAVEL                   0.000000
dtype: float64

In [15]:
nb_name =  "employee attrition challenger"
nb_asset_id = "c6519599-74c5-4656-908e-b39ec4e46b0f"
nb_asset_url = CPD_URL + "/analytics/notebooks/v2/" + nb_asset_id + "?projectid=" + PROJECT_ID + "&context=icp4data"

In [16]:
latestRunId = facts_client.runs.get_current_run_id()
facts_client.runs.log_metric(latestRunId, "Custom accuracy score", CUSTOM_ACC)
facts_client.runs.set_tags(latestRunId, {"Notebook name": nb_name, "Notebook id": nb_asset_id, "Notebook URL" : nb_asset_url})
facts_client.export_facts.export_payload(latestRunId)

2023/04/05 19:42:14 INFO : Initiating logging to factsheet for run_id......64b023d758bc456fa3c5e19a66d09c8f
2023/04/05 19:42:15 INFO : Successfully logged results to Factsheet service for run_id 64b023d758bc456fa3c5e19a66d09c8f under asset_id: e920ab38-e5d4-43d7-8dce-bfc71b9605f0 and project_id : d618d8af-bc48-4582-a228-7f4bcfc327e9


In [17]:
wml_client.set.default_project(PROJECT_ID)

'SUCCESS'

In [18]:
facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

{'name': 'employee attrition challenger',
 'type': 'scikit-learn_1.0',
 'software_spec': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
 'label_column': 'ATTRITION',
 'custom': {'experiment_id': '68a898a987a045afb7b851e39d95d7bc',
  'experiment_name': 'SciKitExperiment_fact'}}

In [19]:
print("Storing model...")
published_model_details = wml_client.repository.store_model(
    model=clf, 
    meta_props=model_props,
    training_target=['ATTRITION'],
    training_data=X)
model_uid = wml_client.repository.get_model_id(published_model_details)

print("Done")
print("Model ID: {}".format(model_uid))

Storing model...
Done
Model ID: 119bcd7d-d3f4-420a-b5d9-3ad4ad1463b9
